# Workshop on Real time video analytics (action recognition)

Course: Real time audio visual sensing and sense making

Website: https://www.iss.nus.edu.sg/executive-education/course/detail/real-time-audio-visual-sensing-and-sense--making/artificial-intelligence

Contact: Tian Jing

Email: tianjing@nus.edu.sg

# Objective
In this workshop, we will perform the following three tasks

- Exercise 1: Perform action recognition using histogram of optical flow
- Exercise 2: Perform action recognition using C3D deep learning approach
- Exercise 3: Perform action recognition using C3D + classifier

# Submission guideline

Once you finish the workshop, rename your .ipynb file to be your name, and submit your .ipynb file into LumiNUS.

In [1]:
import cv2
import os
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import sqrt, pi, arctan2, cos, sin # used for HoF
from scipy.ndimage import uniform_filter # used for hoF
from sklearn import svm

from tensorflow.keras.layers import Activation, Conv3D, Dense, Dropout, Flatten, MaxPooling3D
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix
import tensorflow as tf
# Check GPU coinfiguration in Colab
print("Tensorflow version: ", tf.__version__)
print(tf.test.gpu_device_name())

Tensorflow version:  2.6.0
/device:GPU:0


In [2]:
# Grant access to google drive.
# Run this cell, then you’ll see a link, click on that link, allow access
# Copy the code that pops up, paste it in the box, hit Enter

from google.colab import drive
drive.mount('/content/gdrive')
# Change working directory to be current folder
import os
os.chdir('/content/gdrive/MyDrive/iss/RTAVS/action')
!ls

Mounted at /content/gdrive
 data
 data_test_hof_feature.npz
 data_train_hof_feature.npz
'Day 2 workshop video dataset download link (google drive).txt'
 model_c3d_v0815.h5
 model_c3d_v0815.pt
'wk_action_keras_Apar Garg.ipynb'


# Explore the dataset

- UCF11 Dataset: https://www.crcv.ucf.edu/data/UCF_YouTube_Action.php.

It contains 11 action categories: basketball shooting, biking/cycling, diving, golf swinging, horse back riding, soccer juggling, swinging, tennis swinging, trampoline jumping, volleyball spiking, and walking with a dog.


In [3]:
def load_groups(input_folder):
    '''
    Load the list of sub-folders into a python list with their
    corresponding label.
    '''
    groups         = []
    label_folders  = os.listdir(input_folder)
    index          = 0
    for label_folder in sorted(label_folders):
        label_folder_path = os.path.join(input_folder, label_folder)
        if os.path.isdir(label_folder_path):
            group_folders = os.listdir(label_folder_path)
            for group_folder in group_folders:
                if group_folder != 'Annotation':
                    groups.append([os.path.join(label_folder_path, group_folder), index])
            index += 1

    return groups

#Reference: https://github.com/microsoft/CNTK/blob/master/Examples/Video/DataSets/UCF11/split_ucf11.py
def ucf_split_data(groups, file_ext):
    '''
    Split the data at random for train, eval and test set.
    '''
    group_count = len(groups)
    indices = np.arange(group_count)

    np.random.seed(0) # Make it deterministic.
    np.random.shuffle(indices)

    # 80% training and 20% test.
    train_count = int(0.8 * group_count)
    test_count  = group_count - train_count

    train = []
    test  = []

    for i in range(train_count):
        group = groups[indices[i]]
        video_files = os.listdir(group[0])
        for video_file in video_files:
            video_file_path = os.path.join(group[0], video_file)
            if os.path.isfile(video_file_path):
                video_file_path = os.path.abspath(video_file_path)
                ext = os.path.splitext(video_file_path)[1]
                if (ext == file_ext):
                    train.append([video_file_path, group[1]])

    for i in range(train_count, train_count + test_count):
        group = groups[indices[i]]
        video_files = os.listdir(group[0])
        for video_file in video_files:
            video_file_path = os.path.join(group[0], video_file)
            if os.path.isfile(video_file_path):
                video_file_path = os.path.abspath(video_file_path)
                ext = os.path.splitext(video_file_path)[1]
                if (ext == file_ext):
                    test.append([video_file_path, group[1]])

    return train, test


In [4]:
# Prepare the dataset
ucf_groups = load_groups("data")

ucf_action_labels  = os.listdir("data")
print("action labels: ", ucf_action_labels)

ucf_train, ucf_test = ucf_split_data(ucf_groups, '.avi')
print("Total %d categories, Training data %d sequences, test data %d sequences" % (len(ucf_action_labels), len(ucf_train), len(ucf_test)))

action labels:  ['soccer_juggling', 'basketball', 'trampoline_jumping', 'biking', 'swing', 'volleyball_spiking', 'diving', 'tennis_swing', 'horse_riding', 'golf_swing', 'walking', 'model_c3d_v0815.h5', 'model.hdf5']
Total 13 categories, Training data 1292 sequences, test data 308 sequences


# Exercise 1: Action recognition using histogram of optical flow

- Reference: Histogram of optical flow,  https://github.com/colincsl/pyKinectTools/blob/master/pyKinectTools/algs/HistogramOfOpticalFlow.py

In [5]:
# Reference: https://github.com/colincsl/pyKinectTools/blob/master/pyKinectTools/algs/HistogramOfOpticalFlow.py
# Fix a few bugs
def hof(flow, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), normalise=False, motion_threshold=1.):

    """Extract Histogram of Optical Flow (HOF) for a given image.
    Key difference between this and HOG is that flow is MxNx2 instead of MxN
    Compute a Histogram of Optical Flow (HOF) by
        1. (optional) global image normalisation
        2. computing the dense optical flow
        3. computing flow histograms
        4. normalising across blocks
        5. flattening into a feature vector
    Parameters
    ----------
    Flow : (M, N) ndarray
        Input image (x and y flow images).
    orientations : int
        Number of orientation bins.
    pixels_per_cell : 2 tuple (int, int)
        Size (in pixels) of a cell.
    cells_per_block  : 2 tuple (int,int)
        Number of cells in each block.
    normalise : bool, optional
        Apply power law compression to normalise the image before
        processing.
    static_threshold : threshold for no motion
    Returns
    -------
    newarr : ndarray
        hof for the image as a 1D (flattened) array.
    hof_image : ndarray (if visualise=True)
        A visualisation of the hof image.
    References
    ----------
    * http://en.wikipedia.org/wiki/Histogram_of_oriented_gradients
    * Dalal, N and Triggs, B, Histograms of Oriented Gradients for
      Human Detection, IEEE Computer Society Conference on Computer
      Vision and Pattern Recognition 2005 San Diego, CA, USA
    """
    flow = np.atleast_2d(flow)

    """ 
    -1-
    The first stage applies an optional global image normalisation
    equalisation that is designed to reduce the influence of illumination
    effects. In practice we use gamma (power law) compression, either
    computing the square root or the log of each colour channel.
    Image texture strength is typically proportional to the local surface
    illumination so this compression helps to reduce the effects of local
    shadowing and illumination variations.
    """

    if flow.ndim < 3:
        raise ValueError("Requires dense flow in both directions")

    if normalise:
        flow = sqrt(flow)

    """ 
    -2-
    The second stage computes first order image gradients. These capture
    contour, silhouette and some texture information, while providing
    further resistance to illumination variations. The locally dominant
    colour channel is used, which provides colour invariance to a large
    extent. Variant methods may also include second order image derivatives,
    which act as primitive bar detectors - a useful feature for capturing,
    e.g. bar like structures in bicycles and limbs in humans.
    """

    if flow.dtype.kind == 'u':
        # convert uint image to float
        # to avoid problems with subtracting unsigned numbers in np.diff()
        flow = flow.astype('float')

    gx = np.zeros(flow.shape[:2])
    gy = np.zeros(flow.shape[:2])
    # gx[:, :-1] = np.diff(flow[:,:,1], n=1, axis=1)
    # gy[:-1, :] = np.diff(flow[:,:,0], n=1, axis=0)

    gx = flow[:,:,1]
    gy = flow[:,:,0]


    """ 
    -3-
    The third stage aims to produce an encoding that is sensitive to
    local image content while remaining resistant to small changes in
    pose or appearance. The adopted method pools gradient orientation
    information locally in the same way as the SIFT [Lowe 2004]
    feature. The image window is divided into small spatial regions,
    called "cells". For each cell we accumulate a local 1-D histogram
    of gradient or edge orientations over all the pixels in the
    cell. This combined cell-level 1-D histogram forms the basic
    "orientation histogram" representation. Each orientation histogram
    divides the gradient angle range into a fixed number of
    predetermined bins. The gradient magnitudes of the pixels in the
    cell are used to vote into the orientation histogram.
    """

    magnitude = sqrt(gx**2 + gy**2)
    orientation = arctan2(gy, gx) * (180 / pi) % 180

    sy, sx = flow.shape[:2]
    cx, cy = pixels_per_cell
    bx, by = cells_per_block

    n_cellsx = int(np.floor(sx // cx))  # number of cells in x
    n_cellsy = int(np.floor(sy // cy))  # number of cells in y

    # compute orientations integral images
    orientation_histogram = np.zeros((n_cellsy, n_cellsx, orientations))
    subsample = np.index_exp[int(cy / 2):cy * n_cellsy:cy, int(cx / 2):cx * n_cellsx:cx]
    # There are (orientations-1) bins for optical flow and 1 bin for no-motion
    for i in range(orientations-1):
        #create new integral image for this orientation
        # isolate orientations in this range

        # temp_ori = np.where(orientation < 180 / orientations * (i + 1), orientation, -1)
        # temp_ori = np.where(orientation >= 180 / orientations * i, temp_ori, -1)
        # fixed the bug in the original Github code
        temp_ori = np.where(orientation < 180 / (orientations-1) * (i + 1), orientation, -1)
        temp_ori = np.where(orientation >= 180 / (orientations-1) * i, temp_ori, -1)
        # select magnitudes for those orientations
        cond2 = (temp_ori > -1) * (magnitude > motion_threshold)
        temp_mag = np.where(cond2, magnitude, 0)

        temp_filt = uniform_filter(temp_mag, size=(cy, cx))
        orientation_histogram[:, :, i] = temp_filt[subsample]

    ''' Calculate the no-motion bin '''
    temp_mag = np.where(magnitude <= motion_threshold, magnitude, 0)

    temp_filt = uniform_filter(temp_mag, size=(cy, cx))
    orientation_histogram[:, :, -1] = temp_filt[subsample]

    """
    The fourth stage computes normalisation, which takes local groups of
    cells and contrast normalises their overall responses before passing
    to next stage. Normalisation introduces better invariance to illumination,
    shadowing, and edge contrast. It is performed by accumulating a measure
    of local histogram "energy" over local groups of cells that we call
    "blocks". The result is used to normalise each cell in the block.
    Typically each individual cell is shared between several blocks, but
    its normalisations are block dependent and thus different. The cell
    thus appears several times in the final output vector with different
    normalisations. This may seem redundant but it improves the performance.
    We refer to the normalised block descriptors as Histogram of Oriented
    Gradient (hog) descriptors.
    """

    n_blocksx = (n_cellsx - bx) + 1
    n_blocksy = (n_cellsy - by) + 1
    normalised_blocks = np.zeros((n_blocksy, n_blocksx,
                                  by, bx, orientations))

    for x in range(n_blocksx):
        for y in range(n_blocksy):
            block = orientation_histogram[y:y+by, x:x+bx, :]
            eps = 1e-5
            normalised_blocks[y, x, :] = block / sqrt(block.sum()**2 + eps)

    return normalised_blocks.ravel()

In [6]:
# Define the HoF feature extraction function
def extract_hof_feature(video_list):
    feature_hof = []
    label_list = []
    img_width = 128
    img_height = 64
    for idx, value in enumerate(video_list):
        # Display the progress
        if (idx % 100) == 0:
            print("process sequence %d/%d" % (idx, len(video_list)))
        filename = value[0]
        label = value[1]
        hof_feature_all = []

        cap = cv2.VideoCapture(filename)
        ret, frame = cap.read()
        if ret:
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            gray = cv2.resize(gray, (img_width, img_height)) # Resize frames to reduce feature dimensions
        
            while True:
                previousGray = gray
                ret, frame = cap.read()

                if ret:
                    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                    gray = cv2.resize(gray, (img_width, img_height))
                    flow = cv2.calcOpticalFlowFarneback(previousGray, gray, flow=None, pyr_scale=0.5, levels=5, winsize=11, iterations=10, poly_n=5, poly_sigma=1.1, flags=0)
                    hof_feature_one = hof(flow, orientations=9, pixels_per_cell=(8, 8),cells_per_block=(2, 2))
                    if (len(hof_feature_all) == 0):
                        hof_feature_all = hof_feature_one
                    else:
                        hof_feature_all = np.vstack((hof_feature_all, hof_feature_one))
                else:
                    break
    
        cap.release()
        if (len(hof_feature_all) != 0):
            hof_feature_mean = np.mean(hof_feature_all, axis=0)
            feature_hof.append(hof_feature_mean)
            label_list.append(label)      
        
    return np.array(feature_hof), np.array(label_list)


In [7]:
# It takes around half hour to prepare such feature dataset
# You can uncomment if you want to re-build feature dataset, otherwise, load them from the data files
# The sequence file basketball\v_shooting_24\v_shooting_24_01.avi is very short.

# print("Prepare training feature dataset")
# train_feature_hof, train_label_hof = extract_hof_feature(ucf_train)
# np.savez("data_train_hof_feature.npz", X=train_feature_hof, Y=train_label_hof)
# print(train_feature_hof.shape, train_label_hof.shape)

# print("Prepare test feature dataset")
# test_feature_hof, test_label_hof = extract_hof_feature(ucf_test)
# np.savez("data_test_hof_feature.npz", X=test_feature_hof, Y=test_label_hof)
# print(test_feature_hof.shape, test_label_hof.shape)

In [8]:
# Load HoF features from pre-prepared data file and perform SVM classification
with np.load("data_train_hof_feature.npz") as npzfile:
    x_train_hof = npzfile["X"]
    x_train_hof_label = npzfile["Y"]
    
with np.load("data_test_hof_feature.npz") as npzfile:
    x_test_hof = npzfile["X"]
    x_test_hof_label = npzfile["Y"]
    
print("Training data", x_train_hof.shape, x_train_hof_label.shape)
print("Test data", x_test_hof.shape, x_test_hof_label.shape)

Training data (1293, 3780) (1293,)
Test data (306, 3780) (306,)


In [9]:
hof_svm_model = svm.SVC(kernel = 'linear', C = 10).fit(x_train_hof, x_train_hof_label)

x_test_hof_pred = hof_svm_model.predict(x_test_hof)

print(confusion_matrix(x_test_hof_label, x_test_hof_pred))

[[ 0  1  2  0  0  0  0  1  0  0  1]
 [ 4 34  0  0  4  1  1  0  0  0  1]
 [ 3  0  4  0  0  0  3  0  0  1  2]
 [ 0  0  2 24  1  1  3  1  0  3  1]
 [ 0  0  0  0 17  0  0  0  0  0  2]
 [ 6  0  0 10  0 12  0  2  1  1  0]
 [ 0  6  1  0  0  1 14  0  5  0  1]
 [ 5  0  1  5  4  5  0 24  3  3  2]
 [ 0  2  0  6  0  1  1  0 14  0  0]
 [ 3  0  5  1  2  0  0  1  0  4  2]
 [ 2  4  8  2  1  1  3  0  1  2 10]]


# Exercise 2: Action recognition using C3D model

A modified C3D model is used in the workshop to reduce model training time for demonstration purpose.

- Reference: D. Tran, L. Bourdev, R. Fergus, L. Torresani, and M. Paluri, “Learning Spatiotemporal Features with 3D Convolutional Networks“, ICCV 2015, https://arxiv.org/abs/1412.0767

In [10]:
class Videoto3D:

    def __init__(self, width, height, depth):
        self.width = width
        self.height = height
        self.depth = depth

    def get_data(self, filename, skip=True):
        cap = cv2.VideoCapture(filename)
        nframe = cap.get(cv2.CAP_PROP_FRAME_COUNT)
        bAppend = False
        if (nframe>=self.depth):
            if skip:
                frames = [x * nframe / self.depth for x in range(self.depth)]
            else:
                frames = [x for x in range(self.depth)]
        else:
            print("Insufficient %d frames in video %s, set bAppend as True" % (nframe, filename))
            bAppend = True
            frames = [x for x in range(int(nframe))] # nframe is a float

        framearray = []

        for i in range(len(frames)):#self.depth):
            cap.set(cv2.CAP_PROP_POS_FRAMES, frames[i])
            ret, frame = cap.read()
            frame = cv2.resize(frame, (self.height, self.width))
            framearray.append(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY))

        cap.release()
        
        if bAppend:
            while len(framearray) < self.depth:
                framearray.append(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY))
            print("Append more frames in the framearray to have %d frames" % len(framearray))
                
        return np.array(framearray)

def loaddata(video_list, vid3d, skip=True):
    X = []
    Y = []
    for idx, value in enumerate(video_list):
        # Display the progress
        if (idx % 100) == 0:
            print("process data %d/%d" % (idx, len(video_list)))
        filename = value[0]
        label = value[1]
        Y.append(label)
        X.append(vid3d.get_data(filename, skip=skip))
        
    return np.array(X).transpose((0, 2, 3, 1)), np.array(Y)


In [11]:
# Define parameter setting
class Args:
    batch = 128
    epoch = 50
    nclass = 11 # 11 action categories
    depth = 10
    rows = 32
    cols = 32
    skip = True # Skip: randomly extract frames; otherwise, extract first few frames

param_setting = Args()
img_rows = param_setting.rows
img_cols = param_setting.cols
frames = param_setting.depth
channel = 1
vid3d = Videoto3D(img_rows, img_cols, frames)
nb_classes = param_setting.nclass


In [12]:
# Prepare training data
x_train, y_train = loaddata(ucf_train, vid3d, param_setting.skip)
x_train = x_train.reshape((x_train.shape[0], img_rows, img_cols, frames, channel))
y_train = tf.keras.utils.to_categorical(y_train, nb_classes)  

# Prepare test data
x_test, y_test = loaddata(ucf_test, vid3d, param_setting.skip)
x_test = x_test.reshape((x_test.shape[0], img_rows, img_cols, frames, channel))
y_test = tf.keras.utils.to_categorical(y_test, nb_classes)

print("Training data", x_train.shape, y_train.shape)
print("Test data", x_test.shape, y_test.shape)

process data 0/1292
process data 100/1292
process data 200/1292
process data 300/1292
process data 400/1292
process data 500/1292
process data 600/1292
process data 700/1292
process data 800/1292
process data 900/1292
process data 1000/1292
Insufficient 1 frames in video /content/gdrive/MyDrive/iss/RTAVS/action/data/basketball/v_shooting_24/v_shooting_24_01.avi, set bAppend as True
Append more frames in the framearray to have 10 frames
process data 1100/1292
process data 1200/1292
process data 0/308
process data 100/308
process data 200/308
process data 300/308
Training data (1292, 32, 32, 10, 1) (1292, 11)
Test data (308, 32, 32, 10, 1) (308, 11)


In [13]:
# Define deep learning model
# This is simplified C3D model
c3d_model = Sequential()
c3d_model.add(Conv3D(32, kernel_size=(3, 3, 3), input_shape=(x_train.shape[1:]), padding='same'))
c3d_model.add(Activation('relu'))
c3d_model.add(Conv3D(32, kernel_size=(3, 3, 3), padding='same'))
c3d_model.add(Activation('relu'))
c3d_model.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
c3d_model.add(Dropout(0.2))

c3d_model.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
c3d_model.add(Activation('relu'))
c3d_model.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
c3d_model.add(Activation('relu'))
c3d_model.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
c3d_model.add(Dropout(0.2))

c3d_model.add(Flatten(name='flatten_feature'))
c3d_model.add(Dense(512, activation='relu'))
c3d_model.add(Dropout(0.2))
c3d_model.add(Dense(nb_classes, activation='softmax'))

c3d_model.compile(loss=categorical_crossentropy, optimizer=Adam(), metrics=['accuracy'])
c3d_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 32, 32, 10, 32)    896       
_________________________________________________________________
activation (Activation)      (None, 32, 32, 10, 32)    0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 32, 32, 10, 32)    27680     
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 10, 32)    0         
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 11, 11, 4, 32)     0         
_________________________________________________________________
dropout (Dropout)            (None, 11, 11, 4, 32)     0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 11, 11, 4, 64)     5

In [14]:
# Train the model
c3d_model.fit(x_train, y_train, validation_data=(x_test, y_test), batch_size=param_setting.batch, epochs=param_setting.epoch, verbose=2, shuffle=True)

c3d_model.save_weights("/content/gdrive/MyDrive/iss/RTAVS/action/data/model_c3d_v0815.h5")

Epoch 1/50
11/11 - 36s - loss: 13.4044 - accuracy: 0.0960 - val_loss: 2.4069 - val_accuracy: 0.0487
Epoch 2/50
11/11 - 4s - loss: 2.3974 - accuracy: 0.1068 - val_loss: 2.3984 - val_accuracy: 0.0584
Epoch 3/50
11/11 - 4s - loss: 2.3955 - accuracy: 0.1432 - val_loss: 2.4002 - val_accuracy: 0.0584
Epoch 4/50
11/11 - 4s - loss: 2.3934 - accuracy: 0.1432 - val_loss: 2.4036 - val_accuracy: 0.0584
Epoch 5/50
11/11 - 4s - loss: 2.3904 - accuracy: 0.1385 - val_loss: 2.4075 - val_accuracy: 0.0584
Epoch 6/50
11/11 - 4s - loss: 2.3863 - accuracy: 0.1393 - val_loss: 2.4124 - val_accuracy: 0.0584
Epoch 7/50
11/11 - 4s - loss: 2.3783 - accuracy: 0.1401 - val_loss: 2.4202 - val_accuracy: 0.0584
Epoch 8/50
11/11 - 4s - loss: 2.3519 - accuracy: 0.1393 - val_loss: 2.4793 - val_accuracy: 0.0649
Epoch 9/50
11/11 - 4s - loss: 2.3169 - accuracy: 0.1703 - val_loss: 2.4511 - val_accuracy: 0.0682
Epoch 10/50
11/11 - 4s - loss: 2.2385 - accuracy: 0.2128 - val_loss: 2.4305 - val_accuracy: 0.0942
Epoch 11/50
11/11

In [15]:
c3d_model.save('/content/gdrive/MyDrive/iss/RTAVS/action/data/model.hdf5')

In [16]:
# Load the pre-trained model
c3d_model.load_weights("/content/gdrive/MyDrive/iss/RTAVS/action/data/model_c3d_v0815.h5")

# Evaluate the deep learning model
y_pred = c3d_model.predict(x_test, verbose=0)
print("Confusion matrix")
print(confusion_matrix(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1)))

Confusion matrix
[[ 1  1  1  1  0  0  0  0  0  1  0]
 [ 0 15  2  1 13  3  6  1  2  1  3]
 [ 0  0  6  2  0  0  2  0  1  0  0]
 [ 2  1  0 26  1  0  0  1  1  2  0]
 [ 0  1  0  0 10  0  4  0  1  0  2]
 [ 1  0  0  8  0  0  6  6  6  0  6]
 [ 0  1  2  1  1  2 19  0  2  1  0]
 [ 9  1  1  0  1  7  0 27  1  0  5]
 [ 0  1  0  7  0  9  1  0  3  0  5]
 [ 1  0  0  3  0  0  1  0  3 10  0]
 [ 0  5  0  0  4  3  7  0  2  0 14]]


$\color{red}{\text{Q1: Complete code to perform action recognition using C3D + classifier}}$

Tasks
- Extract the fully connected layer response `flatten_feature` from the C3D model `c3d_model` as features from the training data `x_train`
- Labels are provided in `ucf_train` and `ucf_test`
- Build a classification model, such as SVM
- Perform classification on `x_test`, display the confusion matrix


In [17]:
# Provide your solution to Q1 here
model =load_model('/content/gdrive/MyDrive/iss/RTAVS/action/data/model.hdf5')
model.trainable = False
flatten_layer_output = model.layers[-4].output

new_model = Model(inputs=[model.input], outputs=flatten_layer_output)
new_model.compile()

In [18]:
new_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_input (InputLayer)    [(None, 32, 32, 10, 1)]   0         
_________________________________________________________________
conv3d (Conv3D)              (None, 32, 32, 10, 32)    896       
_________________________________________________________________
activation (Activation)      (None, 32, 32, 10, 32)    0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 32, 32, 10, 32)    27680     
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 10, 32)    0         
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 11, 11, 4, 32)     0         
_________________________________________________________________
dropout (Dropout)            (None, 11, 11, 4, 32)     0     

In [19]:
print("Prepare training feature dataset")
new_x_train = new_model.predict(x_train)
_,new_y_train = loaddata(ucf_train, vid3d, param_setting.skip)
print(new_x_train.shape, new_y_train.shape)

print("Prepare test feature dataset")
new_x_test = new_model.predict(x_test)
_,new_y_test = loaddata(ucf_test, vid3d, param_setting.skip)
print(new_x_test.shape, new_y_test.shape)

Prepare training feature dataset
process data 0/1292
process data 100/1292
process data 200/1292
process data 300/1292
process data 400/1292
process data 500/1292
process data 600/1292
process data 700/1292
process data 800/1292
process data 900/1292
process data 1000/1292
Insufficient 1 frames in video /content/gdrive/MyDrive/iss/RTAVS/action/data/basketball/v_shooting_24/v_shooting_24_01.avi, set bAppend as True
Append more frames in the framearray to have 10 frames
process data 1100/1292
process data 1200/1292
(1292, 2048) (1292,)
Prepare test feature dataset
process data 0/308
process data 100/308
process data 200/308
process data 300/308
(308, 2048) (308,)


In [20]:
svm_model = svm.SVC(kernel = 'linear', C = 10).fit(new_x_train, new_y_train)

new_y_pred = svm_model.predict(new_x_test)

print(confusion_matrix(new_y_pred, new_y_test))

[[ 1  0  0  1  0  3  1  6  1  1  0]
 [ 1 14  0  0  4  0  3  0  3  0  7]
 [ 2  2 10  0  1  2  2  6  4  2  2]
 [ 0  0  0 24  0 17  0  9  2  1  0]
 [ 0 11  0  3 10  0  2  2  0  2  4]
 [ 0  7  0  2  0  1  4  3  8  0  3]
 [ 0  2  1  0  2  2 11  0  0  0  5]
 [ 0  0  0  1  0  4  0 24  1  0  0]
 [ 0  4  0  0  0  3  4  1  2  2  1]
 [ 1  1  0  2  0  0  1  0  1 10  0]
 [ 0  6  0  1  1  1  1  1  4  0 13]]


$\color{red}{\text{Q2: Propose how to apply the model developed in this workshop on the live video streaming.}}$

The model developed in this workshop works on a short video clip. In practice, given a live video streaming, how to apply such model for action recognition?

In [21]:
# Provide your solution to Q2 here (no need programming)
# Take 2 cameras which capture live video stream
# Suppose Camera 1 captures short video sequece. Meanwhile Camera 1 is processing sequence, Camera 2 can capture the scenario.
# Similarly, when Camera 2 is processing, Camera 1 can capture.
# Both cameras capture and process alternately.

**Once you finish the workshop, rename your .ipynb file to be your name, and submit your .ipynb file into LumiNUS.**

Have a nice day!